# "Statistical" metrics

In [1]:
from IPython.display import display, Markdown

When we are dealing with applications that generate text and we want to have more flexibility when evaluating the quality of the generated text, we can use "statistical" metrics.

BLEU and ROUGE are two of the most popular statistical metrics for evaluating the quality of the generated text. 

These metrics work by comparing the generated text to a "golden" reference, their flexibility comes from the fact that it not only checks for specific words (like we did with the rule-based evaluations), but for the entire sequence, and the order of the words, the sentence length, etc.

At the core, BLEU and ROUGE decomposes the generated text into **n-grams** (chunks of words) and compare them to the n-grams of the "golden" reference.


For example, imagine we have the following text:

> Policy Lab has experimented with Artificial Intelligence (AI) in policy development with teams across government, and beyond, for a number of years. In 2019 we worked with the Department for Transport’s data science team to consider the role that AI could play in improving the efficiency and effectiveness of the policy consultation process. In 2022 we used AI to create a vision for the future of Hounslow with the local authority. In 2023, we commissioned the creation of the Ecological Intelligence Agency, a speculative artefact to help experience the role AI might have in future decision-making in environmental policy. 

And we have asked the original writer to provide a summary of the text, **This is our golden reference**:

> Policy Lab has explored the use of AI in policy development across various government projects, including improving policy consultation processes, envisioning future urban planning, and investigating AI's potential role in environmental policy decision-making.

### What we want to evaluate

And we have requested different language models to provide a summary of the text using a variety of prompts.


In [ ]:
reference_summary = "Policy Lab has explored the use of AI in policy development across various government projects, including improving policy consultation processes, envisioning future urban planning, and investigating AI's potential role in environmental policy decision-making."
summary_1 = "Policy Lab has explored the application of Artificial Intelligence in diverse government policy initiatives, including enhancing policy consultation, envisioning local community futures, and examining AI's potential role in environmental policy decisions."
summary_2 = "Policy Lab has explored AI applications in governmental policy creation, collaborating with various agencies to enhance consultation procedures, generate urban forecasts, and envision futuristic environmental decision-making tools."
summary_3 = "Policy Lab has been exploring the application of Artificial Intelligence in various government policy initiatives for several years, including improving policy consultations, envisioning the future of local communities, and speculating on the role of AI in environmental decision-making."
summary_4 = "Policy Lab has been using Artificial Intelligence to replace human decision-making in government policy development, creating automated systems to make key decisions without input from policymakers or the public."

display(Markdown(f"""
## Reference summary\n{reference_summary}\n
## Model 1\n{summary_1}\n
## Model 2\n{summary_2}\n
## Model 3\n{summary_3}\n
## Model 4\n{summary_4}\n
"""))

## BLEU

BLEU stands for Bilingual Evaluation Understudy; and it is a metric that was designed for evaluating the quality of the generated text by comparing it to a set of reference translations.

However, it can be used for other applications, such as evaluating the quality of a generated answer from a RAG pipeline, a summarization task, or even the expected response of a chatbot.


We can then evaluate the quality of the generated summaries using BLEU:

In [3]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
# Perfect match
sentence_bleu(
    [reference_summary.lower().split()],
    reference_summary.lower().split()
)


In [ ]:
sentence_bleu(
    [summary_1.lower().split()],
    reference_summary.lower().split()
)

In [ ]:
sentence_bleu(
    [summary_2.lower().split()],
    reference_summary.lower().split()
)

In [ ]:
sentence_bleu(
    [summary_3.lower().split()],
    reference_summary.lower().split()
)

In [ ]:
sentence_bleu(
    [summary_4.lower().split()],
    reference_summary.lower().split()
)


## ROUGE

ROUGE stands for Recall-Oriented Understudy for Gisting Evaluation. Just like BLEU, it was designed for evaluating the quality of the generated text by comparing it to a set of reference.

In practice ROUGE is actually a set of metrics, each one capturing a different aspect of the generated text, for example:

- ROUGE-1: Unigram
- ROUGE-2: Bigram
- ROUGE-L: Longest Common Subsequence


In [8]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [ ]:
scores = scorer.score(reference_summary, summary_1)
scores

In [ ]:
scores = scorer.score(reference_summary, summary_2)
scores

In [ ]:
scores = scorer.score(reference_summary, summary_3)
scores

In [ ]:
scores = scorer.score(reference_summary, summary_4)
scores
